In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
from jsonpath import jsonpath
import datetime 
from datetime import timedelta ,time, datetime 
import re
from openpyxl import load_workbook
# from msoffcrypto_tool import OfficeFile
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
import glob
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean
import io
pd.set_option('display.max_columns', None)
pd.set_option('display.notebook_repr_html', True)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

# 导入数据

In [88]:
# ymos.status :状态,1:未支付  2:扣款失败 3:已支付 4：已取消 5、申请扣款、6已退款
# tprm.rembursement_status :tprm_还款状态, tprm.overdue_type : 逾期类型,M1-M6
# 
sql = '''-- 租后分期应收实收业务量数据统计  
SELECT  ymos.order_id, ymos.money, ymos.part_payment, ymos.refund_date, ymos.reality_refund_date, ymos.sort, ymos.status 分期租金支付状态,ymos.sesame_promo_money_pay, ymos.promo_money,

tprm.purchase_amount, tprm.all_deposit 总押金, tprm.paid_periods 已还期数, tprm.overdue_periods 逾期期数, tprm.rembursement_status tprm_还款状态, tprm.merchant_name,tprm.model_number,
tprm.now_overdue_days, tprm.commodity_premium 商品溢价费, tprm.overdue_fine 滞纳金, tprm.overdue_type 逾期类型, tprm.specification,tprm.max_overdue_days,

om.status , om.create_time 订单创建时间, om.order_number, om.has_actual, om.order_type, om.order_method,
case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as  status2,
om.order_finish_date as 订单完成时间,

tmu.true_name ,tmu.id_card_num,tmu.mobile,

tod.product_name, tod.new_actual_money, 

cc.name as channel_name_cc,        -- 来源渠道
cc.channel_type_id,              -- 渠道id
pa.name as activity_name,        -- 活动名称
pa.type

,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips  
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.is_vip') end,'"','') as is_vip
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.status') end,'"','') as status_result

from db_rent.ya_merchant_order_stages ymos
left join db_digua_business.t_order om on ymos.order_id = om.id
left join db_digua_business.t_postlease_receivables_monitoring  tprm on ymos.order_id = tprm.order_id
left join  db_digua_business.t_member_user tmu on om.user_id = tmu.id
left join  db_digua_business.t_order_details tod on tprm.order_id = tod.order_id

-- 2025.4.29
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
where  DATE_FORMAT(om.create_time, '%Y-%m-%d') <='2025-07-31'
-- DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2024-12-01'
-- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2025-06-30'
-- and om.status = 4
-- and om.overdue = 0
;
'''

df_zhys = clean.query(sql)
# (27787, 62)
df_zhys.shape

(2592545, 42)

# 剔除商家数据

In [89]:
def drop_sj(df_zhys,col1,col2):
    df_zhys.drop(df_zhys[df_zhys[col1]=="深圳优优大数据科技有限公司"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="优优2店"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="小豚租（代收）"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="苏州蚁诺宝"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="租着用电脑数码"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="北京海鸟窝科技有限公司"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="星晟数码"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="蘑菇时间"].index,inplace=True)

    df_zhys.drop(df_zhys[df_zhys[col1]=="汇客好租"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="澄心优租"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="CPS渠道合作"].index,inplace=True)
    
    df_zhys.drop(df_zhys[df_zhys[col1]=="趣智数码"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="格木木二奢名品"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="广州康基贸易有限公司"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="线下小店"].index,inplace=True)  # 计划
    df_zhys.drop(df_zhys[df_zhys[col1]=="小蚂蚁租机"].index,inplace=True)  #拒量订单
    df_zhys.drop(df_zhys[df_zhys[col1]=="人人享租"].index,inplace=True)  #拒量订单
    df_zhys.drop(df_zhys[df_zhys[col1]=="兴鑫兴通讯"].index,inplace=True)  #二手订单
    df_zhys.drop(df_zhys[df_zhys[col1]=="崇胜数码"].index,inplace=True)  #二手订单
    df_zhys.drop(df_zhys[df_zhys[col1]=="喜卓灵租机"].index,inplace=True)  #拒量订单
    df_zhys.drop(df_zhys[df_zhys[col1]=="喜卓灵新租机"].index,inplace=True)  #拒量订单

    df_zhys.drop(df_zhys[df_zhys[col2].str.contains(pat='探路者',regex=False)==True].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys['merchant_name'] == "乙辉数码"].index, inplace=True)
    return df_zhys

df_zhys = drop_sj(df_zhys,'merchant_name','model_number')
df_zhys.shape

(2592310, 42)

# 处理日期和备注信息

In [90]:
df = df_zhys.copy()
df["下单日期"] = df["订单创建时间"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df["月份"]=df["下单日期"].dt.month
df["下单月份"]=df["订单创建时间"].dt.strftime('%Y-%m')
df['年份'] = df["下单日期"].dt.year

df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")
df["是否进件"] = np.where(df["进件"] == '进件', 1, 0)
df["是否出库"] = np.where(df["status"].isin([2, 3, 4, 5, 6, 8, 15]), 1, 0)
df.loc[:,"来源渠道"]=df["channel_name_cc"].fillna("未知渠道")
df.loc[:,"归属渠道"]=df.apply(lambda x: clean.qudao_type(x["来源渠道"],x["activity_name"],x["order_method"], x['channel_type_id'], x['order_type']),axis=1)

df['归属渠道'] = np.where(df.归属渠道 == '顶部搜索框的搜索结果页', '搜索渠道', df.归属渠道)
# conditions_1 = ['租机宝','八派信息','CPS直播','派金花','富士康租机','九州信息','勉丫租','哈银','租瓜直播','美仑美奂','兔丫租赁','分期乐1','分期乐2','98租超','邦客惠乘车码','推一推','硬派抖音']
conditions_1 = ['八派信息','CPS直播','派金花','九州信息','勉丫租','哈银','租瓜直播','美仑美奂','分期乐1','分期乐2','98租超','推一推','硬派抖音']
# 租机宝 、 富士康租机 、 兔丫租赁 、 邦客惠乘车码  -- 不算S量
df['归属渠道'] = np.where(df.归属渠道.isin(conditions_1), 'S量', df.归属渠道)
df['是否免人审'] = np.where((df.is_vip=='1')&(df.status_result=='0'), 1 , 0)
df['是否拒量'] = np.where(df.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True, 1, 0)
df['是否号卡'] = np.where(df.type == 4, 1, 0)

df["内存"]=df["specification"].str.split("内存：").str[1].str.split("颜色：").str[0].str.split(" ").str[0]
df["颜色"]=df["specification"].str.split("内存：").str[1].str.split("颜色：").str[1].str.split(" ").str[0]

# 去重

In [91]:
df = df[df.是否出库==1]
# 先根据order_id、sort去重
duplicates = df[df.duplicated(subset=['order_id', 'sort'], keep=False)]
if not duplicates.empty:
    print("发现重复数据，数量：", len(duplicates))
    # 可以选择保留最新的记录（根据实际业务逻辑调整）
    # 这里假设reality_refund_date不为空的是更新的记录
    df = df.sort_values(['order_id', 'sort', 'refund_date'], ascending=[True, True, False])
    # 去重，保留每个(order_id, sort)的第一条记录（即最新的）
    df = df.drop_duplicates(subset=['order_id', 'sort'], keep='first')
    

# 订单去重
dict_status_code = {
            "订单取消": 1,
            "待支付": 2,
            "已退款": 3,
            "待审核": 4,
            "待发货": 5,
            "待收货": 6,
            "租赁中": 7,
            "已完成": 8
        }
df["状态编码"] = df["status2"].map(dict_status_code)
df.sort_values(by=["下单日期", "状态编码"], inplace=True)
# 删除重复单号,需要分期数据时不去重order_id
# df.drop_duplicates(subset=["order_id"], inplace=True)
# 删除身份证空值行
df.dropna(subset=["id_card_num"], axis=0, inplace=True)
# 去刷单订单
# df.drop(df[df['total_describes'].str.contains(pat='panli', regex=False) == True].index, inplace=True)
df.drop(df[df['activity_name'] == "1000单秘密计划"].index, inplace=True)
df.drop(df[df['activity_name'] == "1000单秘密计划-无优惠"].index, inplace=True)
df.drop(df[df['activity_name'] == "1000单曙光计划"].index, inplace=True)
df.drop(df[df['activity_name'] == "线下门店3个月试行"].index, inplace=True)
# df.drop_duplicates(subset=["order_id"], inplace=True)
df.drop_duplicates(subset=["true_name", "mobile", "id_card_num", "下单日期","sort"], keep="last", inplace=True)
df.drop(df[df['true_name'].isin(
        ["刘鹏", "谢仕程", "潘立", "洪柳", "陈锦奇", "周杰", "卢腾标", "孔靖", "黄娟", "钟福荣", "邱锐杰", "唐林华"
        , "邓媛斤", "黄子南", "刘莎莎", "赖瑞彤", "孙子文", '淦文豪', '杨明豪', '闫宇龙'])].index, inplace=True)
df.shape

发现重复数据，数量： 408


(749351, 57)

In [72]:
df.columns

Index(['order_id', 'money', 'part_payment', 'refund_date',
       'reality_refund_date', 'sort', '分期租金支付状态', 'purchase_amount', '总押金',
       '已还期数', '逾期期数', 'tprm_还款状态', 'merchant_name', 'model_number',
       'now_overdue_days', '商品溢价费', '滞纳金', '逾期类型', 'specification',
       'max_overdue_days', 'status', '订单创建时间', 'order_number', 'has_actual',
       'order_type', 'order_method', 'status2', '订单完成时间', 'true_name',
       'id_card_num', 'mobile', 'product_name', 'new_actual_money',
       'channel_name_cc', 'channel_type_id', 'activity_name', 'type', 'tips',
       'is_vip', 'status_result', '下单日期', '月份', '下单月份', '年份', '进件', '是否进件',
       '是否出库', '来源渠道', '归属渠道', '是否免人审', '是否拒量', '是否号卡', '内存', '颜色', '状态编码'],
      dtype='object')

In [ ]:
df[df.order_id==8924]

# 订单业务量数据，按订单号统计

In [87]:
# 计算单个订单业务量数据，按订单id聚合计算总应收租金，总已收租金
df.loc[:, '总应收租金'] = df.groupby('order_id')['money'].transform('sum')
df.loc[:, '总已收租金'] = df[df.分期租金支付状态==3].groupby('order_id')['part_payment'].transform('sum')
df.loc[:, '总未收租金'] = df.loc[:, '总应收租金'] - df.loc[:, '总已收租金']
df.loc[:, '总未收租金'] = np.where(df.loc[:, '总未收租金']<0, 0, df.loc[:, '总未收租金'])
df.loc[:, '剩余总未收'] = df['new_actual_money'] - df['总已收租金']
df.loc[:, '剩余总未收'] = np.where(df.loc[:, '剩余总未收']<0, 0, df.loc[:, '剩余总未收'])
# df.head()
# 按订单id去重，保留每个订单id的第一条记录
df = df.drop_duplicates(subset='order_id', keep='first')
# 订单业务量数据：计算所有订单的总应收租金、总已收租金、总未收租金、剩余总未收、purchase_amount、new_actual_money总和

order_business_summary = {
    '总应收租金(万元)': (df['总应收租金'].sum()/ 10000).round(2),
    '总已收租金(万元)': (df['总已收租金'].sum()/ 10000).round(2),
    '总未收租金(万元)': (df['总未收租金'].sum()/ 10000).round(2),
    '剩余总未收(万元)': (df['剩余总未收'].sum()/ 10000).round(2),
    '采购成本总和(万元)': (df['purchase_amount'].sum()/ 10000).round(2),
    '买断价总和(万元)': (df['new_actual_money'].sum()/ 10000).round(2)
}

# 创建一个DataFrame来存储汇总数据
summary_df = pd.DataFrame(list(order_business_summary.items()), columns=['指标', '金额'])
summary_df
# 买断：59754,8702.4400002
# 采购：39025,3323.15
# 总应收租金：32552,0049.28
# 剩余总应收：41017,3532.89000005
# 剩余总应收租金：17593,4053.65000004


,指标,金额
0,总应收租金(万元),41130.56
1,总已收租金(万元),23462.67
2,总未收租金(万元),17178.20
3,剩余总未收(万元),43039.59
4,采购成本总和(万元),43896.36
5,买断价总和(万元),67387.34


In [83]:
df.shape

(56726, 59)

## 何薪华需求

In [24]:
sql_name = '''
SELECT tuvor.order_id, tu.nick_name 分配人, tuvor.update_time 
FROM db_digua_business.t_user_verify_order_record tuvor
left join db_digua_business.t_user tu on tuvor.user_id = tu.id 
where tuvor.del_flag = 0 ORDER BY tuvor.update_time
'''
df_name = clean.query(sql_name)

In [25]:
# 拒量的订单2024年12月到2025年6月正常还款的（租赁中）用户没有逾期的订单，需要订单号，用户下单的手机号码，审核人，还款期数（已还）
df_jl = df[(df.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True)]
# df_jl = df_jl[~df_jl.merchant_name.isin(["小蚂蚁租机","兴鑫兴通讯","人人享租","崇胜数码","喜卓灵租机"])]
# 获取订单已还期数，count(part_payment>0)为已还期数
df_jl.loc['已还期数'] = df_jl.groupby('order_id')['part_payment'].transform(lambda x: (x > 0).sum())
df_jl.shape




(1138, 35)

In [29]:

df_name_new = df_name.sort_values('update_time', ascending=False).groupby('order_id').head(1)
# 内连接，获取出库单分配人
df_jl_name = df_jl.merge(df_name_new, on='order_id', how='inner')
df_jl_name_new = df_jl_name[['order_number', 'mobile', '分配人', '已还期数']]
df_jl_name_new.to_excel('F:/需求/薪华需求/拒量非逾期订单.xlsx', index=False)
# df_jl_name_new.shape